In [32]:
!pip install implicit

In [33]:
import pandas as pd

import numpy as np

from scipy.sparse import csr_matrix, coo_matrix

from implicit.nearest_neighbours import ItemItemRecommender, CosineRecommender, TFIDFRecommender, BM25Recommender
from implicit.evaluation import train_test_split
from implicit.evaluation import precision_at_k, mean_average_precision_at_k, AUC_at_k, ndcg_at_k

In [34]:
data = pd.read_csv('retail_train.csv')

In [35]:
data.head()

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.60,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.00,1631,1,0.0,0.0
2,2375,26984851472,1,1036325,1,0.99,364,-0.30,1631,1,0.0,0.0
3,2375,26984851472,1,1082185,1,1.21,364,0.00,1631,1,0.0,0.0
4,2375,26984851472,1,8160430,1,1.50,364,-0.39,1631,1,0.0,0.0


In [36]:
test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

### 1. Weighted random recommender

In [37]:
def weighted_random_recommendation(items_weights, n=5):
    """Случайные рекоммендации
    
    Input
    -----
    items_weights: pd.DataFrame
        Датафрейм со столбцами item_id, weight. Сумма weight по всем товарам = 1
    """
    
    items = np.array(items_probabilities['item_id'])
    probability = np.array(items_probabilities['probability'])
    
    recs = np.random.choice(items, size=n, replace=False, p=probability)
    
    return recs.tolist()

In [38]:
%%time

data = data_train
items_probabilities = data.groupby('item_id')['sales_value'].sum().reset_index()
items_probabilities['log'] = np.log1p(items_probabilities['sales_value'])
sum_ = items_probabilities.log.sum()
items_probabilities['probability'] = items_probabilities['log']/sum_.sum()
items_probabilities.probability.sum()

Wall time: 173 ms


1.0

In [39]:
weighted_random_recommendation(items_probabilities,5)

[916260, 930388, 9526339, 1109638, 1098846]

### 2. Измерим качество по precision@5

In [40]:
result = data_test.groupby('user_id')['item_id'].unique().reset_index()

In [41]:
result.head()

,user_id,item_id
0,1,"[821867, 834484, 856942, 865456, 889248, 90795..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963..."
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107..."
3,7,"[840386, 889774, 898068, 909714, 929067, 95347..."
4,8,"[835098, 872137, 910439, 924610, 992977, 10412..."


In [42]:
def t(df):
    for j in range(df.shape[1]):
        print(type(df.iloc[0][j]))

In [43]:
def precision_at_k(recommended_list, bought_list, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    bought_list = bought_list
    recommended_list = recommended_list[:k]
    
    flags = np.isin(bought_list, recommended_list)
    
    precision = flags.sum() / len(recommended_list)
    
    return precision

In [44]:
def List(result):
    for i in range(result.shape[0]):
        tt = 'actual'
        result[tt][i] = [int(x) for x in list(result[tt][i].replace('[','').replace(']','').split(', '))]
        tt = 'random_recommendation'
        result[tt][i] = [int(x) for x in list(result[tt][i].replace('[','').replace(']','').split(', '))]
        tt = 'popular_recommendation'
        result[tt][i] = [int(x) for x in list(result[tt][i].replace('[','').replace(']','').split(', '))]
        tt = 'itemitem'
        result[tt][i] = [int(x) for x in list(result[tt][i].replace('[','').replace(']','').split(', '))]
        tt = 'cosine'
        result[tt][i] = [int(x) for x in list(result[tt][i].replace('[','').replace(']','').split(', '))]
        tt = 'tfidf'
        result[tt][i] = [int(x) for x in list(result[tt][i].replace('[','').replace(']','').split(', '))]
        tt = 'own_purchases'
        result[tt][i] = [int(x) for x in list(result[tt][i].replace('[','').replace(']','').split(', '))]

In [45]:
t(result)

<class 'numpy.int64'>
<class 'numpy.ndarray'>


In [ ]:
List(result)